# Binned Likelihood with Energy Dispersion (Python)

The following tutorial shows a way of performing binned likelihood with energy dispersion. Technical details can be found [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Pass8_edisp_usage.html). This tutorial assumes that you've gone through the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) analysis thread. You can also watch a [video tutorial](https://fermi.gsfc.nasa.gov/ssc/data/analysis/video_tutorials/#binned).

# Get the data

 For this thread we use the same data extracted for the [binned likelihood thread](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html) with the following selections:

    Search Center (RA,Dec) = (193.98,-5.82)
    Radius = 15 degrees
    Start Time (MET) = 239557417 seconds (2008-08-04T15:43:37)
    Stop Time (MET) = 302572802 seconds (2010-08-04T00:00:00)
    Minimum Energy = 100 MeV
    Maximum Energy = 500000 MeV

We've provided direct links to the event files as well as the spacecraft data file if you don't want to take the time to use the download server. For more information on how to download LAT data please see the [Extract LAT Data](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/extract_latdata.html) tutorial.

 * L181126210218F4F0ED2738_PH00.fits (5.4 MB)
 * L181126210218F4F0ED2738_PH01.fits (10.8 MB)
 * L181126210218F4F0ED2738_PH02.fits (6.9 MB)
 * L181126210218F4F0ED2738_PH03.fits (9.8 MB)
 * L181126210218F4F0ED2738_PH04.fits (7.8 MB)
 * L181126210218F4F0ED2738_PH05.fits (6.6 MB)
 * L181126210218F4F0ED2738_PH06.fits (4.8 MB)
 * L181126210218F4F0ED2738_SC00.fits (256 MB spacecraft file)


In [1]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH00.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH01.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH02.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH03.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH04.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH05.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH06.fits
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_SC00.fits

--2022-07-13 12:40:28--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH00.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5189760 (4.9M) [application/fits]
Saving to: ‘L181126210218F4F0ED2738_PH00.fits’

L181126210218F4F0ED 100%[===================>]   4.95M   755KB/s    in 7.5s    

2022-07-13 12:40:51 (674 KB/s) - ‘L181126210218F4F0ED2738_PH00.fits’ saved [5189760/5189760]

--2022-07-13 12:40:52--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/L181126210218F4F0ED2738_PH01.fits
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10995840 (10M) [application/fits]
Saving 

In [1]:
!mkdir data
!mv *.fits ./data

You'll first need to make a file list with the names of your input event files.

In [2]:
!ls ./data/*PH*.fits > ./data/binned_events.txt
!cat ./data/binned_events.txt

./data/L181126210218F4F0ED2738_PH00.fits
./data/L181126210218F4F0ED2738_PH01.fits
./data/L181126210218F4F0ED2738_PH02.fits
./data/L181126210218F4F0ED2738_PH03.fits
./data/L181126210218F4F0ED2738_PH04.fits
./data/L181126210218F4F0ED2738_PH05.fits
./data/L181126210218F4F0ED2738_PH06.fits


 In the following analysis we've assumed that you've named your list of data files binned_events.txt. 

# Perform Event Selections

You could follow the unbinned likelihood tutorial to perform your event selections using **gtlike*, **gtmktime**, etc. directly from the command line, and then use pylikelihood later.

But we're going to go ahead and use python. The `gt_apps` module provides methods to call these tools from within python. This'll get us used to using python.

 We first import the gt_apps module to gain access to the tools. 

In [1]:
import gt_apps as my_apps

Now, you can see what objects are part of the gt_apps module by executing:

 Start by running gtselect (called 'filter' in python).

In [2]:
my_apps.filter['evclass'] = 128
my_apps.filter['evtype'] = 3
my_apps.filter['ra'] = 193.98
my_apps.filter['dec'] = -5.82
my_apps.filter['rad'] = 15
my_apps.filter['emin'] = 100
my_apps.filter['emax'] = 300000
my_apps.filter['zmax'] = 90
my_apps.filter['tmin'] = 239557417
my_apps.filter['tmax'] = 302572802
my_apps.filter['infile'] = '@./data/binned_events.txt'
my_apps.filter['outfile'] = './data/3C279_filtered.fits'

Once this is done, we can run gtselect:

In [3]:
my_apps.filter.run()

time -p gtselect infile=@./data/binned_events.txt outfile=./data/3C279_filtered.fits ra=193.98 dec=-5.82 rad=15.0 tmin=239557417.0 tmax=302572802.0 emin=100.0 emax=300000.0 zmin=0.0 zmax=90.0 evclass=128 evtype=3 convtype=-1 phasemin=0.0 phasemax=1.0 evtable="EVENTS" chatter=2 clobber=yes debug=no gui=no mode="ql"
Done.
real 4.26
user 3.91
sys 0.30


 Now, we need to find the GTIs. This is accessed within python via the maketime object: 

In [4]:
my_apps.maketime['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.maketime['filter'] = '(DATA_QUAL>0)&&(LAT_CONFIG==1)'
my_apps.maketime['roicut'] = 'no'
my_apps.maketime['evfile'] = './data/3C279_filtered.fits'
my_apps.maketime['outfile'] = './data/3C279_filtered_gti.fits'

In [5]:
my_apps.maketime.run()

time -p gtmktime scfile=./data/L181126210218F4F0ED2738_SC00.fits sctable="SC_DATA" filter="(DATA_QUAL>0)&&(LAT_CONFIG==1)" roicut=no evfile=./data/3C279_filtered.fits evtable="EVENTS" outfile="./data/3C279_filtered_gti.fits" apply_filter=yes overwrite=no header_obstimes=yes tstart=0.0 tstop=0.0 gtifile="default" chatter=2 clobber=yes debug=no gui=no mode="ql"
real 15.87
user 14.53
sys 1.31


# Livetime and Counts Cubes

## Livetime Cube

 We can now compute the livetime cube. 

In [6]:
my_apps.expCube['evfile'] = './data/3C279_filtered_gti.fits'
my_apps.expCube['scfile'] = './data/L181126210218F4F0ED2738_SC00.fits'
my_apps.expCube['outfile'] = './data/3C279_ltcube.fits'
my_apps.expCube['zmax'] = 90
my_apps.expCube['dcostheta'] = 0.025
my_apps.expCube['binsz'] = 1

In [7]:
my_apps.expCube.run()

time -p gtltcube evfile="./data/3C279_filtered_gti.fits" evtable="EVENTS" scfile=./data/L181126210218F4F0ED2738_SC00.fits sctable="SC_DATA" outfile=./data/3C279_ltcube.fits dcostheta=0.025 binsz=1.0 phibins=0 tmin=0.0 tmax=0.0 file_version="1" zmin=0.0 zmax=90.0 chatter=2 clobber=yes debug=no gui=no mode="ql"
Working on file ./data/L181126210218F4F0ED2738_SC00.fits
.....................!
real 1830.17
user 1826.84
sys 1.04


## Counts Cube

The counts cube is the counts from our data file binned in space and energy. All of the steps above use a circular ROI (or a cone, really). Once you switch to binned analysis, you start doing things in squares. Your counts cube can only be as big as the biggest square that can fit in the circular ROI you already selected. 

In [8]:
my_apps.evtbin['evfile'] = './data/3C279_filtered_gti.fits'
my_apps.evtbin['outfile'] = './data/3C279_ccube.fits'
my_apps.evtbin['scfile'] = 'NONE'
my_apps.evtbin['algorithm'] = 'CCUBE'
my_apps.evtbin['nxpix'] = 100
my_apps.evtbin['nypix'] = 100
my_apps.evtbin['binsz'] = 0.2
my_apps.evtbin['coordsys'] = 'CEL'
my_apps.evtbin['xref'] = 193.98
my_apps.evtbin['yref'] = -5.82
my_apps.evtbin['axisrot'] = 0
my_apps.evtbin['proj'] = 'AIT'
my_apps.evtbin['ebinalg'] = 'LOG'
my_apps.evtbin['emin'] = 100
my_apps.evtbin['emax'] = 500000
my_apps.evtbin['enumbins'] = 37

In [9]:
my_apps.evtbin.run()

time -p gtbin evfile=./data/3C279_filtered_gti.fits scfile=NONE outfile=./data/3C279_ccube.fits algorithm="CCUBE" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 ebinfile=NONE tbinalg="LIN" tbinfile=NONE nxpix=100 nypix=100 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 rafield="RA" decfield="DEC" proj="AIT" hpx_ordering_scheme="RING" hpx_order=3 hpx_ebin=yes hpx_region="" evtable="EVENTS" sctable="SC_DATA" efield="ENERGY" tfield="TIME" chatter=2 clobber=yes debug=no gui=no mode="ql"
This is gtbin version HEAD
gtbin: WARNING: No spacecraft file: EXPOSURE keyword will be set equal to ontime.
real 0.53
user 0.47
sys 0.05


# Exposure Maps
The binned exposure map is an exposure map binned in space and energy. We first need to import the python version of 'gtexpcube2' which doesn't have a gtapp version by default. This is easy to do (you can import any of the command line tools into python this way). Then, you can check out the parameters with the pars function. Here we generate exposure maps for the entire sky. 


In [10]:
from GtApp import GtApp
expCube2= GtApp('gtexpcube2','Likelihood')

In [11]:
expCube2['infile'] = './data/3C279_ltcube.fits'
expCube2['cmap'] = 'none'
expCube2['outfile'] = './data/3C279_BinnedExpMap.fits'
expCube2['irfs'] = 'P8R3_SOURCE_V3'
expCube2['evtype'] = '3'
expCube2['nxpix'] = 1800
expCube2['nypix'] = 900
expCube2['binsz'] = 0.2
expCube2['coordsys'] = 'CEL'
expCube2['xref'] = 193.98
expCube2['yref'] = -5.82
expCube2['axisrot'] = 0
expCube2['proj'] = 'AIT'
expCube2['ebinalg'] = 'LOG'
expCube2['emin'] = 100
expCube2['emax'] = 500000
expCube2['enumbins'] = 37

In [12]:
expCube2.run()

time -p gtexpcube2 infile=./data/3C279_ltcube.fits cmap=none outfile=./data/3C279_BinnedExpMap.fits irfs="P8R3_SOURCE_V3" evtype=3 edisp_bins=0 nxpix=1800 nypix=900 binsz=0.2 coordsys="CEL" xref=193.98 yref=-5.82 axisrot=0.0 proj="AIT" ebinalg="LOG" emin=100.0 emax=500000.0 enumbins=37 ebinfile="NONE" hpx_ordering_scheme="RING" hpx_order=6 bincalc="EDGE" ignorephi=no thmax=180.0 thmin=0.0 table="EXPOSURE" chatter=2 clobber=yes debug=no mode="ql"
Computing binned exposure map....................!
Using evtype=3 (i.e., FRONT/BACK irfs)
real 155.88
user 153.61
sys 2.21


# Compute source maps

The sourcemaps step convolves the LAT response with your source model generating maps for each source in the 
model for use in the likelihood calculation. We use the same [XML file](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml) 
as in the standard [binned likelihood](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/binned_likelihood_tutorial.html)
analysis. Go ahead and download the XML file to your working directory. You will also need the recommended 
models for a normal point source analysis [gll_iem_v07.fits](https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/4fgl/gll_iem_v07.fits) and [iso_P8R3_SOURCE_V3_v1.txt](https://fermi.gsfc.nasa.gov/ssc/data/analysis/software/aux/iso_P8R3_SOURCE_V3_v1.txt). These should already been in your $FERMI_DIR/refdata/fermi/galdiffuse directory.

In [5]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml

--2022-07-13 15:26:12--  https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/BinnedLikelihood/3C279_input_model.xml
Resolving fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)... 129.164.179.26
Connecting to fermi.gsfc.nasa.gov (fermi.gsfc.nasa.gov)|129.164.179.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 184526 (180K) [application/xml]
Saving to: ‘3C279_input_model.xml’

3C279_input_model.x 100%[===================>] 180.20K   328KB/s    in 0.5s    

2022-07-13 15:26:14 (328 KB/s) - ‘3C279_input_model.xml’ saved [184526/184526]



In [6]:
!mv 3C279_input_model.xml data/

In [13]:
my_apps.srcMaps['expcube'] = './data/3C279_ltcube.fits'
my_apps.srcMaps['cmap'] = './data/3C279_ccube.fits'
my_apps.srcMaps['srcmdl'] = './data/3C279_input_model.xml'
my_apps.srcMaps['bexpmap'] = './data/3C279_BinnedExpMap.fits'
my_apps.srcMaps['outfile'] = './data/3C279_srcmap.fits'
my_apps.srcMaps['irfs'] = 'P8R3_SOURCE_V3'
my_apps.srcMaps['evtype'] = '3'

In [14]:
my_apps.srcMaps.run()

time -p gtsrcmaps scfile= sctable="SC_DATA" expcube=./data/3C279_ltcube.fits cmap=./data/3C279_ccube.fits srcmdl=./data/3C279_input_model.xml bexpmap=./data/3C279_BinnedExpMap.fits wmap=none outfile=./data/3C279_srcmap.fits irfs="P8R3_SOURCE_V3" evtype=3 convol=yes resample=yes rfactor=2 minbinsz=0.1 ptsrc=yes psfcorr=yes emapbnds=yes edisp_bins=0 copyall=no chatter=2 clobber=yes debug=no gui=no mode="ql"
Generating SourceMap for 4FGL J1118.2-0415 38....................!
Generating SourceMap for 4FGL J1118.6-1235 38....................!
Generating SourceMap for 4FGL J1119.9-1007 38....................!
Generating SourceMap for 4FGL J1121.3-0011 38....................!
Generating SourceMap for 4FGL J1121.4-0553 38....................!
Generating SourceMap for 4FGL J1122.0-0231 38....................!
Generating SourceMap for 4FGL J1122.5-1440 38....................!
Generating SourceMap for 4FGL J1124.1-1203 38....................!
Generating SourceMap for 4FGL J1124.5-0658 38..........

# Run the Likelihood Analysis

First, import the BinnedAnalysis library. Then, create a likelihood object for the dataset. For more details on the pyLikelihood module, check out the [pyLikelihood Usage Notes](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/python_usage_notes.html). Initially, we will do an analysis without weights

In [15]:
import pyLikelihood
from BinnedAnalysis import *

obs = BinnedObs(srcMaps='./data/3C279_srcmap.fits',binnedExpMap='./data/3C279_BinnedExpMap.fits', expCube='./data/3C279_ltcube.fits',irfs='CALDB')

In [16]:
like = BinnedAnalysis(obs,'./data/3C279_input_model.xml',optimizer='NewMinuit')

Perform the fit and print out the results

In [17]:
likeobj=pyLike.NewMinuit(like.logLike)
like.fit(verbosity=0,covar=True,optObject=likeobj)

73159.31353785182

Check that NewMinuit converged. If you get anything other than '0', then NewMinuit didn't converged.

In [18]:
print(likeobj.getRetCode())

156


Print TS for 3C 279 (4FGL J1256.1-0547)

In [19]:
like.Ts('4FGL J1256.1-0547')

28631.56329077939

Now, we will repeat the likelihood analysis but turning energy dispersion on this time.

In [ ]:
import pyLikelihood
from BinnedAnalysis import *

At this point, you have to create a BinnedConfig object and pass that object to BinnedAnalusis. For the appropriate choice of edisp_bins, please read [this](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Pass8_edisp_usage.html).

In [ ]:
conf = BinnedConfig(edisp_bins=-2)
obs2 = BinnedObs(srcMaps='./data/3C279_srcmap.fits',binnedExpMap='./data/3C279_BinnedExpMap.fits',
expCube='./data/3C279_ltcube.fits',irfs='CALDB')
like2 = BinnedAnalysis(obs2,'./data/3C279_input_model.xml',optimizer='NewMinuit',config=conf)

Perform the fit and print out the results

In [ ]:
likeobj2=pyLike.NewMinuit(like2.logLike)
like2.fit(verbosity=0,covar=True,optObject=likeobj2)

In [ ]:
print(likeobj2.getRetCode())

In [ ]:
like2.Ts('4FGL J1256.1-0547')

After verifying that the fit converged, we see that the TS including energy dispersion is a bit lower than what we found neglecting energy dispersion. The effect is most relevant at energies < 300 MeV, but also induces a smaller systematic offset at higher energies. Please refer to a more complete explanation [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/documentation/Pass8_edisp_usage.html).

In [22]:
likeobj

<pyLikelihood.NewMinuit; proxy of <Swig Object of type 'optimizers::NewMinuit *' at 0x1153123f0> >

In [25]:
help(like.logLike)

Help on BinnedLikelihood in module pyLikelihood object:

class BinnedLikelihood(LogLike)
 |  BinnedLikelihood(*args)
 |  
 |  Proxy of C++ Likelihood::BinnedLikelihood class.
 |  
 |  Method resolution order:
 |      BinnedLikelihood
 |      LogLike
 |      SourceModel
 |      Statistic
 |      Function
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  NpredValue(self, *args)
 |      NpredValue(BinnedLikelihood self, std::string const & srcName, bool weighted=False) -> double
 |      NpredValue(BinnedLikelihood self, std::string const & name, SourceMap srcMap, bool weighted=False) -> double
 |  
 |  __init__(self, *args)
 |      __init__(BinnedLikelihood self, CountsMapBase dataMap, Observation observation, std::string const & srcMapsFile="", bool computePointSources=True, bool applyPsfCorrections=True, bool performConvolution=True, bool resample=True, double resamp_factor=2, double minbinsz=0.1) -> BinnedLikelihood
 |      __init__(BinnedLikelihood self, CountsMapBase da

In [28]:
help(like)

Help on BinnedAnalysis in module BinnedAnalysis object:

class BinnedAnalysis(AnalysisBase.AnalysisBase)
 |  BinnedAnalysis(binnedData, srcModel=None, optimizer='Drmngb', use_bl2=False, verbosity=0, psfcorr=True, wmap=None, config=None, delete_local_fixed=False)
 |  
 |  Method resolution order:
 |      BinnedAnalysis
 |      AnalysisBase.AnalysisBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, binnedData, srcModel=None, optimizer='Drmngb', use_bl2=False, verbosity=0, psfcorr=True, wmap=None, config=None, delete_local_fixed=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __setitem__(self, name, value)
 |  
 |  addSource(self, src, binnedConfig=None)
 |      Add a source to the active model.  You should pass a source
 |      object to this function.  Example:
 |      
 |      > test_src = pyLike.PointSource(0, 0, like.observation.observation)
 |      > pl = pyLike.SourceFactory_funcFactory().create("PowerLaw")
 | 

In [29]:
help(pyLike.NewMinuit)

Help on class NewMinuit in module pyLikelihood:

class NewMinuit(Optimizer)
 |  NewMinuit(*args)
 |  
 |  Proxy of C++ optimizers::NewMinuit class.
 |  
 |  Method resolution order:
 |      NewMinuit
 |      Optimizer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Minos(self, n, level=1.0, numericDeriv=False)
 |      Minos(NewMinuit self, unsigned int n, double level=1., bool numericDeriv=False) -> DoublePair
 |  
 |  MnContour(self, par1, par2, level=1.0, npts=20)
 |      MnContour(NewMinuit self, unsigned int par1, unsigned int par2, double level=1., unsigned int npts=20)
 |  
 |  __init__(self, *args)
 |      __init__(NewMinuit self, Statistic arg2) -> NewMinuit
 |      __init__(NewMinuit self, NewMinuit x) -> NewMinuit
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  covarianceMatrix(self)
 |      covarianceMatrix(NewMinuit self) -> DoubleVectorVector
 |  
 |  find_min(self, *args)
 |      find_min(NewMinuit self, int verbose=0, double tole=1e-5, int tolType=ABSOLUTE)

In [34]:
like.logLike

<pyLikelihood.BinnedLikelihood; proxy of <Swig Object of type 'Likelihood::BinnedLikelihood *' at 0x1150e58d0> >

In [35]:
dir(like)

['NpredValue',
 'Ts',
 'Ts_old',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_errors',
 '_importPlotter',
 '_inputs',
 '_isDiffuseOrNearby',
 '_minosIndexError',
 '_npredValues',
 '_plotData',
 '_plotResiduals',
 '_plotSource',
 '_renorm',
 '_separation',
 '_setSourceAttributes',
 '_srcCnts',
 '_srcDialog',
 '_ts_src',
 '_xrange',
 'addGaussianPrior',
 'addPrior',
 'addPriors',
 'addSource',
 'binnedData',
 'constrain_norms',
 'constrain_params',
 'covar_is_current',
 'covariance',
 'deleteSource',
 'e_vals',
 'energies',
 'energyFlux',
 'energyFluxError',
 'fit',
 'flux',
 'fluxError',
 'freePars',
 'freeze',
 'getExtr

In [41]:
help(like.params)

Help on method params in module AnalysisBase:

params() method of BinnedAnalysis.BinnedAnalysis instance
    Returns a list of all of the parameters in the active
    model.



In [44]:
like.params()[0]

 Prefactor:  2.340e+00  0.000e+00  0.000e+00  1.000e+02 ( 9.025e-13) fixed

In [48]:
help(like.setSpectrum)

Help on method setSpectrum in module AnalysisBase:

setSpectrum(srcName, functionName) method of BinnedAnalysis.BinnedAnalysis instance
    Set the spectral shape of a source identified by "srcName".
    The "functionName" is a string that must be one of the
    avaialble spectral functions (i.e. PowerLaw, PowerLaw2 etc.).
    You can also pass a spectrum object (see the addSource
    docString for more details).  The spectral parameters of this
    new model will be set to defaults which are probably not what
    you want them to be.

